In [1]:
import pandas as pd


In [ ]:
# --- 1. MEMBACA DAN MEMPERSIAPKAN DATA ---
print("1. Membaca file data...")
try:
    # Ganti nama file jika berbeda
    df = pd.read_csv("dataset/data_cuaca_historis_bandung.csv") 
    # df = pd.read_csv("data_cuaca_augmented.csv") 
except FileNotFoundError:
    print("Error: File 'data_cuaca_historis_bandung.csv' tidak ditemukan.")
    exit()

1. Membaca file data...


In [3]:
# Mengubah 'timestamp' menjadi tipe data datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"Data berhasil dibaca. Jumlah baris: {len(df)}")
print("Contoh data:")
print(df.head())

Data berhasil dibaca. Jumlah baris: 131
Contoh data:
                   timestamp   suhu  kelembaban  kecepatan_angin  arah_angin  \
0 2025-09-22 10:34:02.191570  28.65          64             1.86          29   
1 2025-09-22 10:35:02.325098  28.65          64             1.86          29   
2 2025-09-22 10:36:02.547758  28.65          64             1.86          29   
3 2025-09-22 10:37:02.696813  28.65          64             1.86          29   
4 2025-09-22 10:38:02.839180  28.65          64             1.86          29   

   tekanan_udara  intensitas_hujan  
0           1012               0.0  
1           1012               0.0  
2           1012               0.0  
3           1012               0.0  
4           1012               0.0  


In [4]:
def tentukan_kelas_prediksi(row, langkah_prediksi):
    # Cek apakah akan hujan dalam 'langkah_prediksi' ke depan
    akan_hujan = row['intensitas_hujan_mendatang'] > 0
    
    if not akan_hujan:
        return 0 # Kelas 0: Tidak Hujan
    
    # Jika akan hujan, tentukan arahnya dari 'arah_angin' saat ini
    arah_angin = row['arah_angin']
    
    if 337.5 <= arah_angin < 360 or 0 <= arah_angin < 22.5:
        return 1 # Utara
    elif 22.5 <= arah_angin < 67.5:
        return 2 # Timur Laut
    elif 67.5 <= arah_angin < 112.5:
        return 3 # Timur
    elif 112.5 <= arah_angin < 157.5:
        return 4 # Tenggara
    elif 157.5 <= arah_angin < 202.5:
        return 5 # Selatan
    elif 202.5 <= arah_angin < 247.5:
        return 6 # Barat Daya
    elif 247.5 <= arah_angin < 292.5:
        return 7 # Barat
    else: # 292.5 <= arah_angin < 337.5
        return 8 # Barat Laut

# --- Di dalam skrip latih_model.py Anda ---
# Ganti blok Feature Engineering Anda dengan ini:

# ... (setelah membaca df)
langkah_prediksi = 15
df['intensitas_hujan_mendatang'] = df['intensitas_hujan'].shift(-langkah_prediksi).rolling(window=langkah_prediksi).max()
df.dropna(inplace=True)

# Buat target multi-kelas baru
df['target_prediksi'] = df.apply(lambda row: tentukan_kelas_prediksi(row, langkah_prediksi), axis=1)

# Hapus kolom bantuan
df.drop(columns=['intensitas_hujan_mendatang'], inplace=True)

# Target (y) sekarang adalah kolom baru ini
y = df['target_prediksi']
# df.sort_values('target_prediksi').reset_index(drop=False)
# Tambahkan baris ini untuk melihat jumlah data per kelas
print("Mengecek distribusi kelas pada variabel y:")
print(y.value_counts())


Mengecek distribusi kelas pada variabel y:
target_prediksi
0    102
Name: count, dtype: int64
